<a href="https://colab.research.google.com/github/diegomartinmendez/CEIA_PLN_1/blob/main/Desafio_1_Diego_Mendez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facultad de Ingeniería - Universidad de Buenos Aires
## Laboratorio de Sistemas Embebidos
## Carrera de Especialización en Inteligencia Artificial
## Procesamiento Natual del Lenguaje I
### 2° Bimestre 2025 - Cohorte 17
### Mg. Ing. Diego Martín Méndez

## Desafío 1
### Punto 1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_20newsgroups

#### Carga de Datos

In [2]:
# Cargo los datos (ya separados de forma predeterminada en train y test):
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# Compruebo que el elemento está en formato de sklearn:
type(newsgroups_train)

sklearn.utils._bunch.Bunch

In [4]:
# Cantidad de documentos en train:
print(len(newsgroups_train.data))

11314


In [5]:
# Cantidad de documentos en test:
print(len(newsgroups_test.data))

7532


In [6]:
# Imprimo un documento:
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [7]:
# Imprimo otro documento:
print(newsgroups_train.data[345])

Davis will be paid by three clubs this year, I think the Phils are
responsbible for about $600,000 or so.  They didn't wait for him to clear
waivers as three other clubs were also very interested in him.  A gamble?
Yes.

Won the CY Young, too, for that year.


#### Vectorización

In [8]:
tfidfvect = TfidfVectorizer()

In [9]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [10]:
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [11]:
# La cantidad de documentos coincide con len(newsgroups_train.data).

In [12]:
tfidfvect.vocabulary_['president']

73039

In [13]:
# Diccionario opuesto que va de índices a términos:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [14]:
list(idx2word.items())[:10]

[(95844, 'was'),
 (97181, 'wondering'),
 (48754, 'if'),
 (18915, 'anyone'),
 (68847, 'out'),
 (88638, 'there'),
 (30074, 'could'),
 (37335, 'enlighten'),
 (60560, 'me'),
 (68080, 'on')]

In [15]:
# Primeros 10 términos:
list(sorted(idx2word.items()))[:10]

[(0, '00'),
 (1, '000'),
 (2, '0000'),
 (3, '00000'),
 (4, '000000'),
 (5, '00000000'),
 (6, '0000000004'),
 (7, '00000000b'),
 (8, '00000001'),
 (9, '00000001b')]

In [16]:
# Ültimos 10 términos:
list(sorted(idx2word.items()))[-10:]

[(101621, 'zznkzz'),
 (101622, 'zznp'),
 (101623, 'zzrk'),
 (101624, 'zzy_3w'),
 (101625, 'zzz'),
 (101626, 'zzzoh'),
 (101627, 'zzzzzz'),
 (101628, 'zzzzzzt'),
 (101629, '³ation'),
 (101630, 'ýé')]

In [17]:
# en y_train guardo los targets que son enteros:
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [18]:
# Hay 20 clases correspondientes a los 20 grupos de noticias:
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [19]:
# Elejo 5 índices aleatorios únicos:
indices = random.sample(range(len(newsgroups_train.data)), 5)

# Muestro los 5 elementos aleatorios:
for idx in indices:
    print(f"Título: {newsgroups_train.filenames[idx]}")
    print(f"Texto: {newsgroups_train.data[idx][:500]}...")     # muestro solo los primeros 500 caracteres
    print("-" * 80)

Título: /root/scikit_learn_data/20news_home/20news-bydate-train/sci.crypt/15602
Texto: 
: Do you know of any freely distributable c++ (or c) code for public
: key cryptography (such as RSA)?  

: I've tried various archie searches to no avail.  

Have you heard of PGP? I assume from your post that you have not. PGP 2.2
is a freeware RSA encryption program which includes digital signatures and
comprehensive key management facilities. Most sites also keep the source code.
A growing number of people are using this excellent software to encrypt (to
a very high standard) their email an...
--------------------------------------------------------------------------------
Título: /root/scikit_learn_data/20news_home/20news-bydate-train/misc.forsale/76020
Texto: Hi,

	I have a Quantum ProDrive LPS 40 MB SCSI hard drive for sale.
	It came with my MacIIsi and was replaced by a larger hard drive.
	In great working condition. Fast and quiet. Never had a problem.

	Asking $100+COD shipping or reasonab

In [20]:
# Recorro cada documento elegido:
for idx in indices:
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    print(f"\n Documento índice {idx} - Categoría: {newsgroups_train.target_names[newsgroups_train.target[idx]]}")

    # Ordeno los índices por similaridad (descendente), excluyendo el propio documento:
    similar_indices = cossim.argsort()[::-1][1:6]

    print("Top 5 documentos más similares:")
    for i in similar_indices:
        categoria = newsgroups_train.target_names[newsgroups_train.target[i]]
        print(f"  Índice: {i}, Similaridad: {cossim[i]:.4f}, Categoría: {categoria}")


 Documento índice 10154 - Categoría: sci.crypt
Top 5 documentos más similares:
  Índice: 1138, Similaridad: 0.5130, Categoría: sci.crypt
  Índice: 2899, Similaridad: 0.4153, Categoría: sci.crypt
  Índice: 10861, Similaridad: 0.3760, Categoría: sci.crypt
  Índice: 5407, Similaridad: 0.3531, Categoría: sci.crypt
  Índice: 6005, Similaridad: 0.3296, Categoría: sci.crypt

 Documento índice 8883 - Categoría: misc.forsale
Top 5 documentos más similares:
  Índice: 3725, Similaridad: 0.2729, Categoría: comp.sys.ibm.pc.hardware
  Índice: 6727, Similaridad: 0.2573, Categoría: comp.sys.mac.hardware
  Índice: 7722, Similaridad: 0.2517, Categoría: comp.sys.ibm.pc.hardware
  Índice: 4882, Similaridad: 0.2382, Categoría: comp.sys.ibm.pc.hardware
  Índice: 1913, Similaridad: 0.2378, Categoría: comp.sys.ibm.pc.hardware

 Documento índice 3121 - Categoría: alt.atheism
Top 5 documentos más similares:
  Índice: 2043, Similaridad: 0.2946, Categoría: soc.religion.christian
  Índice: 10575, Similaridad: 0.2

### Punto 2

In [21]:
# Inicio un modelo de clasificación Naïve Bayes y lo entrenar con sklearn:
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [22]:
# Con el vectorizador ya fiteado en train, vectorizo los textos del conjunto de test:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# El F1-score es una metrica adecuada para reportar desempeño de modelos de claificación,
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados.
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

In [24]:
def entrenar_modelos_nb(vectorizer_params=None):

  newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
  newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

  # Parámetros por defecto si no se pasan
  if vectorizer_params is None:
      vectorizer_params = {
          'norm': 'l2',
          'use_idf': True,
          'smooth_idf': True,
          'sublinear_tf': False,
          }

  # Crear vectorizador y transformar datos
  tfidfvect = TfidfVectorizer(**vectorizer_params)
  X_train = tfidfvect.fit_transform(newsgroups_train.data)
  y_train = newsgroups_train.target
  X_test = tfidfvect.transform(newsgroups_test.data)
  y_test = newsgroups_test.target

  # Entrenar modelo
  clf = MultinomialNB()
  clf.fit(X_train, y_train)

  # Predicción y evaluación
  y_pred = clf.predict(X_test)
  f1 = f1_score(y_test, y_pred, average='macro')
  print(f"F1-score macro: {f1:.4f}")
  return f1

In [25]:
params = {
    'norm': 'l2',
    'use_idf': True,
    'smooth_idf': True,
    'sublinear_tf': False,
    }

In [26]:
entrenar_modelos_nb(vectorizer_params=params)

F1-score macro: 0.5854


0.5854345727938506

In [27]:
from itertools import product

# Espacio de búsqueda para cada parámetro
param_grid = {
    'norm': ['l1', 'l2'],
    'use_idf': [True, False],
    'smooth_idf': [True, False],
    'sublinear_tf': [True, False]
}

In [28]:
# Generar todas las combinaciones de parámetros
param_names = list(param_grid.keys())
param_values = list(param_grid.values())

combinaciones = list(product(*param_values))

In [29]:
"""
resultados = []

for combinacion in combinaciones:
    params = dict(zip(param_names, combinacion))
    print(f"Probando combinación: {params}")
    f1 = entrenar_modelos_nb(vectorizer_params=params)
    resultados.append((params, f1))
"""

'\nresultados = []\n\nfor combinacion in combinaciones:\n    params = dict(zip(param_names, combinacion))\n    print(f"Probando combinación: {params}")\n    f1 = entrenar_modelos_nb(vectorizer_params=params)\n    resultados.append((params, f1))\n'

In [30]:
def entrenar_modelos_nb(vectorizer_class=TfidfVectorizer, vectorizer_params=None,
                        model_class=MultinomialNB, model_params=None):

    # Cargar datos
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
    newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

    # Si no se pasan parámetros, usar los por defecto de la clase correspondiente
    if vectorizer_params is None:
        vectorizer_params = {
            'norm': 'l2',
            'use_idf': True,
            'smooth_idf': True,
            'sublinear_tf': False,
        }

    if model_params is None:
        model_params = {
            'alpha': 1.0,
            'force_alpha': True,
            'fit_prior': True,
            'norm': False
        }

    # Mostrar configuración utilizada
    print(f"Modelo: {model_class.__name__}")
    print(f"Parámetros del modelo:\n{model_params}")
    print(f"\n Vectorizador: {vectorizer_class.__name__}")
    print(f"Parámetros del vectorizador:\n{vectorizer_params}\n")

    # Instanciar y aplicar vectorizador
    vectorizer = vectorizer_class(**vectorizer_params)
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    X_test = vectorizer.transform(newsgroups_test.data)
    y_train = newsgroups_train.target
    y_test = newsgroups_test.target

    # Instanciar y entrenar modelo
    clf = model_class(**model_params)
    clf.fit(X_train, y_train)

    # Predicción y evaluación
    y_pred = clf.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f"F1-score macro obtenido: {f1:.4f}")
    return f1


In [31]:
# Pruebo un ejemplo:

entrenar_modelos_nb(
    vectorizer_params={
        'ngram_range': (1, 2),
        'min_df': 3,
        'use_idf': True,
        'norm': 'l1'
    },
    model_class=ComplementNB,
    model_params={
        'alpha': 0.5,
        'force_alpha': True,
        'fit_prior': False,
        'norm': False
    }
)

Modelo: ComplementNB
Parámetros del modelo:
{'alpha': 0.5, 'force_alpha': True, 'fit_prior': False, 'norm': False}

 Vectorizador: TfidfVectorizer
Parámetros del vectorizador:
{'ngram_range': (1, 2), 'min_df': 3, 'use_idf': True, 'norm': 'l1'}

F1-score macro obtenido: 0.6614


0.6613543916778575

In [32]:
# Genero los parámetros con los cuales voy a evaluar los modelos:

from itertools import product

# Espacio de búsqueda:

# Parámetros de vectorizador:
vectorizer_grid = {
    'norm': ['l1', 'l2'],
    'use_idf': [True, False],
    'smooth_idf': [True, False],
    'sublinear_tf': [True, False],
}

# Modelos y sus respectivos grids:
model_grids = {
    MultinomialNB: {
        'alpha': [0.1, 1.0],
        'force_alpha': [True, False],
        'fit_prior': [True, False]
    },
    ComplementNB: {
        'alpha': [0.1, 1.0],
        'force_alpha': [True, False],
        'fit_prior': [True, False],
        'norm': [False, True]  # este parámetro solo existe en ComplementNB
    }
}

In [33]:
resultados = []

# Combinaciones de parámetros de vectorizador:
vec_param_names = list(vectorizer_grid.keys())
vec_param_values = list(vectorizer_grid.values())
vec_combinations = list(product(*vec_param_values))

# Recorre los modelos:
for model_class, model_grid in model_grids.items():
    model_param_names = list(model_grid.keys())
    model_param_values = list(model_grid.values())
    model_combinations = list(product(*model_param_values))

    for vec_vals in vec_combinations:
        vectorizer_params = dict(zip(vec_param_names, vec_vals))

        for model_vals in model_combinations:
            model_params = dict(zip(model_param_names, model_vals))

            print("\n==============================")
            print(f"Probando combinación:")
            print(f"Modelo: {model_class.__name__}")
            print(f"Vectorizador: {vectorizer_params}")
            print(f"Modelo params: {model_params}")
            print("==============================")

            # Ejecuta la función entrenar_modelos_nb:
            try:
                f1 = entrenar_modelos_nb(
                    vectorizer_class=TfidfVectorizer,
                    vectorizer_params=vectorizer_params,
                    model_class=model_class,
                    model_params=model_params
                )
                resultados.append((model_class.__name__, vectorizer_params, model_params, f1))
            except Exception as e:
                print(f"Error en esta combinación: {e}")


Se truncaron las últimas líneas 5000 del resultado de transmisión.
Parámetros del modelo:
{'alpha': 1.0, 'force_alpha': False, 'fit_prior': False}

 Vectorizador: TfidfVectorizer
Parámetros del vectorizador:
{'norm': 'l2', 'use_idf': True, 'smooth_idf': True, 'sublinear_tf': True}

F1-score macro obtenido: 0.6025

Probando combinación:
Modelo: MultinomialNB
Vectorizador: {'norm': 'l2', 'use_idf': True, 'smooth_idf': True, 'sublinear_tf': False}
Modelo params: {'alpha': 0.1, 'force_alpha': True, 'fit_prior': True}
Modelo: MultinomialNB
Parámetros del modelo:
{'alpha': 0.1, 'force_alpha': True, 'fit_prior': True}

 Vectorizador: TfidfVectorizer
Parámetros del vectorizador:
{'norm': 'l2', 'use_idf': True, 'smooth_idf': True, 'sublinear_tf': False}

F1-score macro obtenido: 0.6565

Probando combinación:
Modelo: MultinomialNB
Vectorizador: {'norm': 'l2', 'use_idf': True, 'smooth_idf': True, 'sublinear_tf': False}
Modelo params: {'alpha': 0.1, 'force_alpha': True, 'fit_prior': False}
Modelo:

In [34]:
# Ordena por f1-score descendente:
resultados_ordenados = sorted(resultados, key=lambda x: x[3], reverse=True)

print("\n Mejores combinaciones:")
for i, (modelo, vect, modparams, f1) in enumerate(resultados_ordenados[:5], 1):
    print(f"{i}) Modelo: {modelo}, F1: {f1:.4f}")
    print(f"   Vectorizador: {vect}")
    print(f"   Modelo params: {modparams}")


 Mejores combinaciones:
1) Modelo: ComplementNB, F1: 0.6998
   Vectorizador: {'norm': 'l2', 'use_idf': False, 'smooth_idf': True, 'sublinear_tf': True}
   Modelo params: {'alpha': 0.1, 'force_alpha': True, 'fit_prior': True, 'norm': False}
2) Modelo: ComplementNB, F1: 0.6998
   Vectorizador: {'norm': 'l2', 'use_idf': False, 'smooth_idf': True, 'sublinear_tf': True}
   Modelo params: {'alpha': 0.1, 'force_alpha': True, 'fit_prior': False, 'norm': False}
3) Modelo: ComplementNB, F1: 0.6998
   Vectorizador: {'norm': 'l2', 'use_idf': False, 'smooth_idf': True, 'sublinear_tf': True}
   Modelo params: {'alpha': 0.1, 'force_alpha': False, 'fit_prior': True, 'norm': False}
4) Modelo: ComplementNB, F1: 0.6998
   Vectorizador: {'norm': 'l2', 'use_idf': False, 'smooth_idf': True, 'sublinear_tf': True}
   Modelo params: {'alpha': 0.1, 'force_alpha': False, 'fit_prior': False, 'norm': False}
5) Modelo: ComplementNB, F1: 0.6998
   Vectorizador: {'norm': 'l2', 'use_idf': False, 'smooth_idf': False, 

### Punto 3

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
# 1. Vectorizo el corpus completo:
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)     # limitado a 10k palabras
X = vectorizer.fit_transform(fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes')).data)

# 2. Transpongo la matriz documento-término → término-documento:
X_T = X.T  # shape: (n_terms, n_docs)

# 3. Obtengo el vocabulario como lista de palabras:
terms = np.array(vectorizer.get_feature_names_out())

In [37]:
# Palabras elegidas:
palabras_objetivo = ['space', 'god', 'car', 'windows', 'computer']

In [38]:
for palabra in palabras_objetivo:
    if palabra not in terms:
        print(f"Palabra '{palabra}' no encontrada en el vocabulario.")
        continue

    idx = np.where(terms == palabra)[0][0]
    vector_palabra = X_T[idx]

    # Similaridad con todas las demás palabras
    sim = cosine_similarity(vector_palabra, X_T)[0]

    # Ordenar: de mayor a menor (excepto la misma palabra)
    indices_similares = sim.argsort()[::-1][1:6]
    palabras_similares = terms[indices_similares]

    print(f"\n Palabra: '{palabra}'")
    for i, similar in enumerate(palabras_similares):
        print(f"  {i+1}. {similar} (sim={sim[indices_similares[i]]:.4f})")


 Palabra: 'space'
  1. nasa (sim=0.3142)
  2. shuttle (sim=0.2745)
  3. exploration (sim=0.2249)
  4. aeronautics (sim=0.2131)
  5. launch (sim=0.2094)

 Palabra: 'god'
  1. jesus (sim=0.2733)
  2. christ (sim=0.2638)
  3. bible (sim=0.2610)
  4. faith (sim=0.2514)
  5. existence (sim=0.2394)

 Palabra: 'car'
  1. cars (sim=0.1932)
  2. dealer (sim=0.1688)
  3. civic (sim=0.1507)
  4. loan (sim=0.1501)
  5. owner (sim=0.1391)

 Palabra: 'windows'
  1. dos (sim=0.3109)
  2. ms (sim=0.2274)
  3. microsoft (sim=0.2073)
  4. file (sim=0.1926)
  5. nt (sim=0.1917)

 Palabra: 'computer'
  1. wu (sim=0.1283)
  2. graphics (sim=0.1126)
  3. drive (sim=0.1065)
  4. reboot (sim=0.1046)
  5. hackers (sim=0.1031)
